In [115]:
import requests
import env
import os
import pandas as pd

In [24]:
from env import Client_ID

In [25]:
from igdb.wrapper import IGDBWrapper

### Connecting to API

In [91]:
url = env.get_db_url()
response = requests.post(url)
print(response.text)


{"access_token":"vuoie67yp79e34aiyhbt4t4sxzwwp7","expires_in":5518646,"token_type":"bearer"}



In [92]:
data = response.json()
data.keys()

dict_keys(['access_token', 'expires_in', 'token_type'])

In [93]:
access_token = data['access_token']

In [94]:
access_token

'vuoie67yp79e34aiyhbt4t4sxzwwp7'

### Making requests from API

- Most of the requests to the API will use the POST method
- The base URL is: https://api.igdb.com/v4
- You define which endpoint you wish to query by appending /{endpoint name} to the base URL eg. https://api.igdb.com/v4/games
- Include Client ID and access_token in the HEADER of your request so that your headers look like the following. Take special care of the capitalisation. Bearer should be hard-coded.

    ''' 
    Client-ID: Client ID

    Authorization: Bearer access_token 
    '''

- You use the BODY of your request to specify the fields you want to retrieve as well as any other filters, sorting etc

**Example**

    '''
    POST https://api.igdb.com/v4/games
    Client-ID: abcdefg12345
    Authorization: Bearer access12345token
    fields *;
    '''

more examples https://api-docs.igdb.com/#examples 

In [37]:
url = env.get_db_url()
base_url = 'https://api.igdb.com/v4'
response = requests.post(base_url)
print(response.text)

{
  "message": "Authorization Failure. Have you tried:",
  "Tip 1":   "Ensure you are sending Authorization and Client-ID as headers.",
  "Tip 2":   "Ensure Authorization value starts with 'Bearer ', including the space",
  "Tip 3":   "Ensure Authorization value ends with the App Access Token you generated, NOT your Client Secret.",
  "Docs":    "https://api-docs.igdb.com/#authentication",
  "Discord": "https://discord.gg/FrvfwQg"
}



In [68]:
url = env.get_db_url()
headers = {'Client-ID': f'{Client_ID}', 'Authorization': f'Bearer {access_token}'}
base_url = 'https://api.igdb.com/v4'
response = requests.post(base_url , headers=headers)
print(response.text)

{"message":"'1amjgiwxks20vysq6dq1hqi23vlwyf' not a valid key=value pair (missing equal-sign) in Authorization header: 'Bearer 1amjgiwxks20vysq6dq1hqi23vlwyf'."}


In [70]:
headers = {'Client-ID': f'{Client_ID}', 'Authorization': f'Bearer {access_token}'}
data = 'fields *; popularity;sort popularity desc;'
r = requests.post('https://api.igdb.com/v4/games' ,data=data ,  headers=headers)

In [71]:
r.json()

[{'id': 202489,
  'artworks': [77476],
  'category': 1,
  'cover': 224545,
  'created_at': 1653310572,
  'external_games': [2442147, 2443444],
  'genres': [15, 16],
  'name': 'Panzer Corps 2: Axis Operations - 1943',
  'parent_game': 27716,
  'screenshots': [798986,
   798987,
   798988,
   798989,
   798990,
   798991,
   798992,
   798993,
   798994],
  'similar_games': [13196,
   18623,
   28244,
   32510,
   36258,
   55036,
   79134,
   102057,
   102163,
   112588],
  'slug': 'panzer-corps-2-axis-operations-1943',
  'summary': 'The critical hour where the fate of Germany on the Eastern Front will be decided has arrived! Though the Battle of Stalingrad has ended, the Axis forces in Russia remain in turmoil.',
  'tags': [22, 268435471, 268435472],
  'themes': [22],
  'updated_at': 1654096076,
  'url': 'https://www.igdb.com/games/panzer-corps-2-axis-operations-1943',
  'videos': [68880],
  'websites': [285491, 286755],
  'checksum': '2992a43b-4d59-7ab8-e500-71a352404759'},
 {'id': 4

In [69]:
response.json()



{'message': "'1amjgiwxks20vysq6dq1hqi23vlwyf' not a valid key=value pair (missing equal-sign) in Authorization header: 'Bearer 1amjgiwxks20vysq6dq1hqi23vlwyf'."}

In [64]:
r.text

'[\n  {\n    "id": 202489,\n    "artworks": [\n      77476\n    ],\n    "category": 1,\n    "cover": 224545,\n    "created_at": 1653310572,\n    "external_games": [\n      2442147,\n      2443444\n    ],\n    "genres": [\n      15,\n      16\n    ],\n    "name": "Panzer Corps 2: Axis Operations - 1943",\n    "parent_game": 27716,\n    "screenshots": [\n      798986,\n      798987,\n      798988,\n      798989,\n      798990,\n      798991,\n      798992,\n      798993,\n      798994\n    ],\n    "similar_games": [\n      13196,\n      18623,\n      28244,\n      32510,\n      36258,\n      55036,\n      79134,\n      102057,\n      102163,\n      112588\n    ],\n    "slug": "panzer-corps-2-axis-operations-1943",\n    "summary": "The critical hour where the fate of Germany on the Eastern Front will be decided has arrived! Though the Battle of Stalingrad has ended, the Axis forces in Russia remain in turmoil.",\n    "tags": [\n      22,\n      268435471,\n      268435472\n    ],\n    "th

In [75]:
headers = {'Client-ID': f'{Client_ID}', 'Authorization': f'Bearer {access_token}'}
data = 'fields *; rating; sort rating asc;'
r = requests.post('https://api.igdb.com/v4/games', data=data, headers=headers)

https://www.igdb.com/games/grand-theft-auto-v-special-edition

headers = {"Authorization": f"token {github_token}", "User-Agent": github_username}

if headers["Authorization"] == "token " or headers["User-Agent"] == "":
    raise Exception(
        "You need to follow the instructions marked TODO in this script before trying to use it"
    )

In [56]:
wrapper = IGDBWrapper(f'{Client_ID}', f'{access_token}')

In [76]:
'''With a wrapper instance already created'''
# JSON API request
byte_array = wrapper.api_request(
            'games',
            'fields *; limit 500;'
          )
# parse into JSON however you like...

# Protobuf API request
from igdb.igdbapi_pb2 import GameResult
byte_array = wrapper.api_request(
            'games.pb', # Note the '.pb' suffix at the endpoint
            'fields *; limit 500;'
          )
games_message = GameResult()
games_message.ParseFromString(byte_array) # Fills the protobuf message object with the response

235695

In [127]:
byte_array = wrapper.api_request(
            'games',
            'fields *; limit 500;'
          )

In [111]:
byte_array

b'[\n  {\n    "id": 202489,\n    "artworks": [\n      77476\n    ],\n    "category": 1,\n    "cover": 224545,\n    "created_at": 1653310572,\n    "external_games": [\n      2442147,\n      2443444\n    ],\n    "genres": [\n      15,\n      16\n    ],\n    "name": "Panzer Corps 2: Axis Operations - 1943",\n    "parent_game": 27716,\n    "screenshots": [\n      798986,\n      798987,\n      798988,\n      798989,\n      798990,\n      798991,\n      798992,\n      798993,\n      798994\n    ],\n    "similar_games": [\n      13196,\n      18623,\n      28244,\n      32510,\n      36258,\n      55036,\n      79134,\n      102057,\n      102163,\n      112588\n    ],\n    "slug": "panzer-corps-2-axis-operations-1943",\n    "summary": "The critical hour where the fate of Germany on the Eastern Front will be decided has arrived! Though the Battle of Stalingrad has ended, the Axis forces in Russia remain in turmoil.",\n    "tags": [\n      22,\n      268435471,\n      268435472\n    ],\n    "t

In [128]:
import json
y = json.loads(byte_array)

In [129]:
pd.DataFrame(y)

,id,artworks,category,cover,created_at,external_games,genres,name,parent_game,screenshots,...,collection,franchises,follows,bundles,franchise,aggregated_rating,aggregated_rating_count,multiplayer_modes,dlcs,ports
0,202489,[77476],1,224545.0,1653310572,"[2442147, 2443444]","[15, 16]",Panzer Corps 2: Axis Operations - 1943,27716.0,"[798986, 798987, 798988, 798989, 798990, 79899...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,40104,NaN,0,NaN,1498435200,"[20127, 1988884]",[5],Dogou Souken,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,85031,NaN,0,61466.0,1517392813,"[216805, 1969751, 2005770]",[9],City Mysteries,NaN,"[155025, 155026, 155027]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,99234,NaN,0,91582.0,1524843905,"[603512, 1857496, 1917109]","[31, 32]",Sword of the Black Stone,NaN,"[231528, 231529, 231530, 231531, 231532]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,148930,[33736],0,169077.0,1621959514,"[2052375, 2057300]","[13, 15, 31, 32]",Bok-Bok: A Chicken Dating Sim,NaN,"[463145, 463148, 463149, 463151, 463152]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,23731,NaN,0,17178.0,1473206400,[1983600],[33],Flappy Meatbag,NaN,"[31405, 31406, 31407, 31408]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
496,76350,NaN,0,110494.0,1511784381,"[190853, 196961, 1191097]","[5, 32]",Season's Beatings,NaN,"[136760, 136761, 136762, 136763, 136764]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
497,109623,NaN,0,NaN,1537526466,"[1600980, 1980624]",[32],TheTruth.exe,NaN,"[255193, 255194, 255195, 255196, 255197]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
498,115515,NaN,0,90900.0,1551028489,"[1710258, 1805872, 1916137]","[31, 32]",My Beautiful Paper Smile,NaN,"[289889, 289890, 289891, 289892, 289893, 28989...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [130]:
pd.DataFrame(y).columns

Index(['id', 'artworks', 'category', 'cover', 'created_at', 'external_games',
       'genres', 'name', 'parent_game', 'screenshots', 'similar_games', 'slug',
       'summary', 'tags', 'themes', 'updated_at', 'url', 'videos', 'websites',
       'checksum', 'first_release_date', 'platforms', 'release_dates',
       'age_ratings', 'involved_companies', 'game_modes',
       'player_perspectives', 'alternative_names', 'game_engines', 'keywords',
       'status', 'rating', 'rating_count', 'storyline', 'total_rating',
       'total_rating_count', 'version_parent', 'version_title', 'hypes',
       'collection', 'franchises', 'follows', 'bundles', 'franchise',
       'aggregated_rating', 'aggregated_rating_count', 'multiplayer_modes',
       'dlcs', 'ports'],
      dtype='object')

In [120]:
pd.DataFrame(y['id', 'category', 'genres', 'name', 'parent_game', 'slug','summary', 'tags', 'themes', 'url', 'websites',])

TypeError: list indices must be integers or slices, not tuple

In [139]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [141]:

pd.DataFrame(y).head(1)

,id,artworks,category,cover,created_at,external_games,genres,name,parent_game,screenshots,similar_games,slug,summary,tags,themes,updated_at,url,videos,websites,checksum,first_release_date,platforms,release_dates,age_ratings,involved_companies,game_modes,player_perspectives,alternative_names,game_engines,keywords,status,rating,rating_count,storyline,total_rating,total_rating_count,version_parent,version_title,hypes,collection,franchises,follows,bundles,franchise,aggregated_rating,aggregated_rating_count,multiplayer_modes,dlcs,ports
0,202489,[77476],1,224545.0,1653310572,"[2442147, 2443444]","[15, 16]",Panzer Corps 2: Axis Operations - 1943,27716.0,"[798986, 798987, 798988, 798989, 798990, 79899...","[13196, 18623, 28244, 32510, 36258, 55036, 791...",panzer-corps-2-axis-operations-1943,The critical hour where the fate of Germany on...,"[22, 268435471, 268435472]",[22],1654096076,https://www.igdb.com/games/panzer-corps-2-axis...,[68880],"[285491, 286755]",2992a43b-4d59-7ab8-e500-71a352404759,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [135]:
pd.DataFrame(y)

,id,artworks,category,cover,created_at,external_games,genres,name,parent_game,screenshots,...,collection,franchises,follows,bundles,franchise,aggregated_rating,aggregated_rating_count,multiplayer_modes,dlcs,ports
0,202489,[77476],1,224545.0,1653310572,"[2442147, 2443444]","[15, 16]",Panzer Corps 2: Axis Operations - 1943,27716.0,"[798986, 798987, 798988, 798989, 798990, 79899...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,40104,NaN,0,NaN,1498435200,"[20127, 1988884]",[5],Dogou Souken,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,85031,NaN,0,61466.0,1517392813,"[216805, 1969751, 2005770]",[9],City Mysteries,NaN,"[155025, 155026, 155027]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,99234,NaN,0,91582.0,1524843905,"[603512, 1857496, 1917109]","[31, 32]",Sword of the Black Stone,NaN,"[231528, 231529, 231530, 231531, 231532]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,148930,[33736],0,169077.0,1621959514,"[2052375, 2057300]","[13, 15, 31, 32]",Bok-Bok: A Chicken Dating Sim,NaN,"[463145, 463148, 463149, 463151, 463152]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,23731,NaN,0,17178.0,1473206400,[1983600],[33],Flappy Meatbag,NaN,"[31405, 31406, 31407, 31408]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
496,76350,NaN,0,110494.0,1511784381,"[190853, 196961, 1191097]","[5, 32]",Season's Beatings,NaN,"[136760, 136761, 136762, 136763, 136764]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
497,109623,NaN,0,NaN,1537526466,"[1600980, 1980624]",[32],TheTruth.exe,NaN,"[255193, 255194, 255195, 255196, 255197]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
498,115515,NaN,0,90900.0,1551028489,"[1710258, 1805872, 1916137]","[31, 32]",My Beautiful Paper Smile,NaN,"[289889, 289890, 289891, 289892, 289893, 28989...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [113]:
y

[{'id': 202489,
  'artworks': [77476],
  'category': 1,
  'cover': 224545,
  'created_at': 1653310572,
  'external_games': [2442147, 2443444],
  'genres': [15, 16],
  'name': 'Panzer Corps 2: Axis Operations - 1943',
  'parent_game': 27716,
  'screenshots': [798986,
   798987,
   798988,
   798989,
   798990,
   798991,
   798992,
   798993,
   798994],
  'similar_games': [13196,
   18623,
   28244,
   32510,
   36258,
   55036,
   79134,
   102057,
   102163,
   112588],
  'slug': 'panzer-corps-2-axis-operations-1943',
  'summary': 'The critical hour where the fate of Germany on the Eastern Front will be decided has arrived! Though the Battle of Stalingrad has ended, the Axis forces in Russia remain in turmoil.',
  'tags': [22, 268435471, 268435472],
  'themes': [22],
  'updated_at': 1654096076,
  'url': 'https://www.igdb.com/games/panzer-corps-2-axis-operations-1943',
  'videos': [68880],
  'websites': [285491, 286755],
  'checksum': '2992a43b-4d59-7ab8-e500-71a352404759'}]

In [78]:
games_message.ParseFromString(byte_array)

235695

In [88]:
type(games_message)

igdbapi_pb2.GameResult

In [90]:
games_message[1]

TypeError: 'GameResult' object is not subscriptable

get ratings

In [104]:
headers = {'Client-ID': f'{Client_ID}', 'Authorization': f'Bearer {access_token}'}
#data = 'fields *; popularity;sort popularity desc;'
requests.get('https://api.igdb.com/v4/age_ratings' ,  headers=headers).text

'[\n  {\n    "id": 22474\n  },\n  {\n    "id": 43430\n  },\n  {\n    "id": 2853\n  },\n  {\n    "id": 32550\n  },\n  {\n    "id": 65037\n  },\n  {\n    "id": 10\n  },\n  {\n    "id": 65004\n  },\n  {\n    "id": 13683\n  },\n  {\n    "id": 78460\n  },\n  {\n    "id": 70534\n  }\n]'